In [2]:
!pip3 install biopython

import Bio.Blast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 16.0 MB/s eta 0:00:00


In [38]:
def find_mismatches(seq1, seq2, seq3):
    """
    Finds the indices where two sequences differ and returns the mismatched characters from seq1.

    Args:
        seq1: The first sequence.
        seq2: The second sequence.

    Returns:
        A tuple containing:
            - A list of indices where the sequences differ.
            - A list of the corresponding characters from seq1 at the mismatch indices.
        Returns None if the sequences have different lengths.
    """
    if len(seq1) != len(seq2):
        print("Sequences must have the same length.")
        return None

    mismatches = []
    mismatch_chars_seq3 = []
    muts =[]
    for i in range(len(seq1)):
        if seq1[i] != seq2[i]:
            mismatches.append(i)
            mismatch_chars_seq3.append(seq3[i])  # Store the character from seq1
            num=i+1
            muts.append(f"{seq1[i]}{num}{seq3[i]}")
    return mismatches, mismatch_chars_seq3,muts



In [73]:
# Example Usage
#sequence1 = "MSMTDLLSAEDIKKAIGAFTAADSFDHKKFFQMVGLKKKSADDVKKVFHILDKDKSGFIEEDELGSILKGFSSDARDLSAKETKTLMAAGDKDGDGKIGVEEFSTLVAES"
#sequence2 = "MSMTDLLSAEDIKKAIGAF AADSFDHKKFFQMVGLKKKS DDVKKVFHILDKDKSGFIEEDELGSILKGFSSDARDLSAKETK L+AAGDKDGDGKIGVEEFSTLVAES"
#sequence3 = "MSMTDLLSAEDIKKAIGAFAAADSFDHKKFFQMVGLKKKSPDDVKKVFHILDKDKSGFIEEDELGSILKGFSSDARDLSAKETKMLLAAGDKDGDGKIGVEEFSTLVAES"
#result = find_mismatches(sequence1, sequence2, sequence3)

def doit(result):
  print("1RWY omits the first residue, M, therefore cutting this off here")
  query = result['query'][1:]
  match = result['match'][1:]
  subject = result['subject'][1:]
  print(query)
  print(subject)
  mismatches = find_mismatches(query, match,subject)

  if mismatches:
      mismatch_indices, mismatch_chars_seq1,muts = mismatches
      #print("Indices where sequences differ:")
      #print(mismatch_indices)
      #print("Corresponding characters from sequence1:")
      #print(mismatch_chars_seq1)
      print(muts)
  else:
      print("Sequences are identical.")

  return muts

In [35]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

def run_blast_and_extract_data(fasta_sequence, database="pdb", program="blastp", evalue=0.01, hitlist_size=10):
    """
    Runs a BLAST search using NCBIWWW and extracts alignment data.

    Args:
        fasta_sequence: The input FASTA sequence.
        database: The BLAST database to search against (e.g., "pdb", "nr", "swissprot").
        program: The BLAST program to use (e.g., "blastp", "blastn", "blastx").
        evalue: The E-value threshold for the BLAST search.
        hitlist_size: The number of top hits to retrieve.

    Returns:
        A list of dictionaries, where each dictionary represents an HSP
        and contains the query sequence, subject sequence, match line,
        bit score, and other alignment information.
        Returns None if an error occurs.
    """
    try:
        result_handle = NCBIWWW.qblast(program, database, fasta_sequence, expect=evalue, hitlist_size=hitlist_size)
        blast_records = NCBIXML.parse(result_handle)

        extracted_data = []
        for blast_record in blast_records:
            for alignment in blast_record.alignments:
                for hsp in alignment.hsps:
                    hsp_data = {
                        "query": hsp.query,
                        "match": hsp.match,
                        "subject": hsp.sbjct,
                        "bits": hsp.bits,
                        "score": hsp.score,
                        "evalue": hsp.expect,
                        "identities": hsp.identities,
                        "positives": hsp.positives,
                        "gaps": hsp.gaps,
                        "query_start": hsp.query_start,
                        "query_end": hsp.query_end,
                        "subject_start": hsp.sbjct_start,
                        "subject_end": hsp.sbjct_end,
                    }
                    extracted_data.append(hsp_data)
        return extracted_data

    except Exception as e:
        print(f"An error occurred: {e}")
        return None



## SLOW

In [ ]:
# Example Usage
hitlist_size=100
fasta_sequence = "MSMTDLLSAEDIKKAIGAFTAADSFDHKKFFQMVGLKKKSADDVKKVFHILDKDKSGFIEEDELGSILKGFSSDARDLSAKETKTLMAAGDKDGDGKIGVEEFSTLVAES"
#blast_results = run_blast_and_extract_data(fasta_sequence, database="swissprot", program="blastp", evalue=0.01, hitlist_size=hitlist_size)
blast_results = run_blast_and_extract_data(fasta_sequence, database="nr", program="blastp", evalue=0.01, hitlist_size=hitlist_size)

In [76]:
allmuts = []
if blast_results:
    for i, result in enumerate(blast_results):
        print(f"HSP {i + 1}:")
        print(f"  Query:   {result['query']}")
        print(f"  Match:   {result['match']}")
        print(f"  Subject: {result['subject']}")
        print(f"  Bits:    {result['bits']}")
        #print(f"  Score:   {result['score']}")
        #print(f"  Evalue:  {result['evalue']}")
        #print(f"  Identities: {result['identities']}")
        #print(f"  Positives:  {result['positives']}")
        #print(f"  Gaps:       {result['gaps']}")
        #print(f"  Query Start:  {result['query_start']}")
        #print(f"  Query End:    {result['query_end']}")
        #print(f"  Subject Start:{result['subject_start']}")
        #print(f"  Subject End:  {result['subject_end']}")
        if 'MSMTDLL' in result['query']:
          print("1RWY omits the first residue, M, therefore cutting this off here")
          muts = doit(result)
          allmuts = allmuts + muts
        else:
          print("Skipping")
        print("-" * 20)
else:
    print("BLAST search or data extraction failed.")

HSP 1:
  Query:   MSMTDLLSAEDIKKAIGAFTAADSFDHKKFFQMVGLKKKSADDVKKVFHILDKDKSGFIEEDELGSILKGFSSDARDLSAKETKTLMAAGDKDGDGKIGVEEFSTLVAES
  Match:   MSMTDLLSAEDIKKAIGAFTAADSFDHKKFFQMVGLKKKSADDVKKVFHILDKDKSGFIEEDELGSILKGFSSDARDLSAKETKTLMAAGDKDGDGKIGVEEFSTLVAES
  Subject: MSMTDLLSAEDIKKAIGAFTAADSFDHKKFFQMVGLKKKSADDVKKVFHILDKDKSGFIEEDELGSILKGFSSDARDLSAKETKTLMAAGDKDGDGKIGVEEFSTLVAES
  Bits:    209.149
1RWY omits the first residue, M, therefore cutting this off here
1RWY omits the first residue, M, therefore cutting this off here
SMTDLLSAEDIKKAIGAFTAADSFDHKKFFQMVGLKKKSADDVKKVFHILDKDKSGFIEEDELGSILKGFSSDARDLSAKETKTLMAAGDKDGDGKIGVEEFSTLVAES
SMTDLLSAEDIKKAIGAFTAADSFDHKKFFQMVGLKKKSADDVKKVFHILDKDKSGFIEEDELGSILKGFSSDARDLSAKETKTLMAAGDKDGDGKIGVEEFSTLVAES
[]
--------------------
HSP 2:
  Query:   MSMTDLLSAEDIKKAIGAFTAADSFDHKKFFQMVGLKKKSADDVKKVFHILDKDKSGFIEEDELGSILKGFSSDARDLSAKETKTLMAAGDKDGDGKIGVEEFSTLVAES
  Match:   MSMTD+LSAEDIKKAIGAF AADSFDHKKFFQMVGLKKK+ D+VKKVFHILDKDKSGFIEEDELGSILKGFSSDARDLSAKETKTL+AAGDKDG

In [77]:
interfaces = [11,15,18,19,20,21,22,23,24,26,29,33,67,70,75,78,80,81,85,88]
muts = doit(result)
for interfaceRes in interfaces:
  interfaceRes = str(interfaceRes)
  for entry in allmuts:
    if interfaceRes in entry:
        print(entry)


1RWY omits the first residue, M, therefore cutting this off here
VKKVFHILDKDKSGFIEEDELGSILKGFSSDARDLSAKETKTLMAAGDKDGDGKIGVEEFSTLVA
LRAAFDVYDVDGDGRITAAELGKVLGRIGEG---CSAEECERMIASVDVDGDGCVGFEEFKKMMC
['V1L', 'K2R', 'K3A', 'V4A', 'H6D', 'I7V', 'L8Y', 'K10V', 'K12G', 'S13D', 'F15R', 'E17T', 'E18A', 'D19A', 'S23K', 'I24V', 'K26G', 'G27R', 'F28I', 'S29G', 'S30E', 'D31G', 'A32-', 'R33-', 'D34-', 'L35C', 'K38E', 'T40C', 'K41E', 'T42R', 'L43M', 'M44I', 'A46S', 'G47V', 'K49V', 'K54C', 'I55V', 'V57F', 'S61K', 'T62K', 'L63M', 'V64M', 'A65C']
I11V
I11V
I11V
I11V
I11V
I15V
I15V
I15V
I15V
I15V
I15L
I15L
I15L
I15L
I15L
I15L
I15L
I15L
I15L
I15L
I15L
I15L
I15L
I15L
I15L
I15L
I15L
I15L
I15P
F18C
F18C
F18C
F18C
F18C
F18C
F18C
F18C
F18C
F18C
F18C
F18C
F18C
F18C
F18C
F18C
F18C
F18M
T19A
T19A
T19S
T19S
T19A
T19A
T19K
T19Q
T19Q
T19Q
T19R
T19Q
T19Q
T19Q
T19Q
T19X
T19K
T19K
T19A
T19A
T19K
T19K
T19K
T19K
T19R
A20D
A20D
A20D
A20D
A20D
A20S
A20S
A20H
A21I
A21T
A21V
A21V
A21P
A21P
A21P
A21P
A21P
A21P
A21P
A21X
A21E


In [56]:
def check_if_100_in_entries(input_list):
    """
    Checks if the number '100' is present in any of the entries of a list of strings.

    This function considers '100' to be present if the *substring* '100'
    is found within any of the strings in the list.

    Args:
        input_list: A list of strings.

    Returns:
        True if the substring '100' is found in any entry, False otherwise.
    """
    for entry in input_list:
        if '100' in entry:
            return True
    return False

# Example Usage (using the same list as before, but you can change it)
my_list = ['S1H', 'L5V', 'S7P', 'E9G', 'K12S', 'I15V', 'G16E', 'T19A', 'A21P', 'D25N',
           'Q31E', 'V33C', 'K37S', 'S39G', 'A40P', 'D42V', 'V43M', 'K45Q', 'H48G', 'K52Q',
           'K54R', 'G64C', 'S65L', 'I66M', 'S71T', 'S72P', 'D73N', 'D76S', 'A79V', 'K83T',
           'T84A', 'M86L', 'V99M', 'E100D', 'S103V', 'A107S']

# Check if '100' is in any of the entries
result = check_if_100_in_entries(my_list)

# Print the result
if result:
    print("The substring '100' is found in at least one entry.")
else:
    print("The substring '100' is not found in any entry.")

The substring '100' is found in at least one entry.
